## PHẦN 1: TẢI VÀ XỬ LÝ DỮ LIỆU

In [22]:
# 1. Cài đặt thư viện
!pip install -U torch==2.3.1 torchtext==0.18.0 spacy portalocker
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

import torch
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pad_sequence
import io
import os
import torch.nn as nn
import random
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


# Kiểm tra phiên bản
print(f"Torch version: {torch.__version__}")
print(f"Torchtext version: {torchtext.__version__}")

# Tải dataset Multi30K (en-fr)
!mkdir -p data
!wget -q https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/train.en.gz -O data/train.en.gz
!wget -q https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/train.fr.gz -O data/train.fr.gz
!wget -q https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/val.en.gz -O data/val.en.gz
!wget -q https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/val.fr.gz -O data/val.fr.gz
!wget -q https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/test_2016_flickr.en.gz -O data/test.en.gz
!wget -q https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/test_2016_flickr.fr.gz -O data/test.fr.gz

# Giải nén
!gunzip -kf data/*.gz
!ls -la data/

print("✅ Đã chuẩn bị xong dữ liệu và thư viện!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 63.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Torch version: 2.3.1+cu121
Torchtext version: 0.18.0+cpu
total 5352
drwxr-xr-x 2 root root    4096 Dec 11 18:30 .
drwxr-xr-x 1 root root    4096 Dec 11 17:53 ..
-rw-

In [23]:
# --- BƯỚC PHỤ: KIỂM TRA DỮ LIỆU (DÙNG ĐỂ VIẾT BÁO CÁO) ---

def read_data(en_file, fr_file):
    with open(en_file, 'r', encoding='utf-8') as f:
        en_sentences = [line.strip() for line in f.readlines() if line.strip()]
    with open(fr_file, 'r', encoding='utf-8') as f:
        fr_sentences = [line.strip() for line in f.readlines() if line.strip()]
    return en_sentences, fr_sentences

# Đọc dữ liệu train, val, test từ folder 'data/'
train_en, train_fr = read_data('data/train.en', 'data/train.fr')
val_en, val_fr = read_data('data/val.en', 'data/val.fr')
test_en, test_fr = read_data('data/test.en', 'data/test.fr')

print("="*30)
print("THỐNG KÊ DỮ LIỆU ")
print("="*30)
print(f" Train: {len(train_en)} cặp câu ")
print(f" Validation: {len(val_en)} cặp câu ")
print(f" Test: {len(test_en)} cặp câu ")

# Hiển thị ví dụ
print("\n---  Ví dụ 5 cặp câu đầu tiên ---")
for i in range(5):
    print(f"Example {i+1}:")
    print(f" - EN: {train_en[i]}")
    print(f" - FR: {train_fr[i]}")
    print("-" * 20)

THỐNG KÊ DỮ LIỆU 
 Train: 29000 cặp câu 
 Validation: 1014 cặp câu 
 Test: 1000 cặp câu 

---  Ví dụ 5 cặp câu đầu tiên ---
Example 1:
 - EN: Two young, White males are outside near many bushes.
 - FR: Deux jeunes hommes blancs sont dehors près de buissons.
--------------------
Example 2:
 - EN: Several men in hard hats are operating a giant pulley system.
 - FR: Plusieurs hommes en casque font fonctionner un système de poulies géant.
--------------------
Example 3:
 - EN: A little girl climbing into a wooden playhouse.
 - FR: Une petite fille grimpe dans une maisonnette en bois.
--------------------
Example 4:
 - EN: A man in a blue shirt is standing on a ladder cleaning a window.
 - FR: Un homme dans une chemise bleue se tient sur une échelle pour nettoyer une fenêtre.
--------------------
Example 5:
 - EN: Two men are at the stove preparing food.
 - FR: Deux hommes aux fourneaux préparent à manger.
--------------------


In [24]:
# --- CẤU HÌNH TOKEN ĐẶC BIỆT (<unk>, <pad>, <sos>, <eos>) ---

UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
SPECIAL_SYMBOLS = ['<unk>', '<pad>', '<sos>', '<eos>']

# 1. HÀM: get_tokenizers()
# Nhiệm vụ: Tải và trả về object tokenizer của spacy cho tiếng Anh và Pháp.

def get_tokenizers():
    try:
        en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
        fr_tokenizer = get_tokenizer('spacy', language='fr_core_news_sm')
        print(" Đã tải thành công Tokenizer (Spacy).")
        return en_tokenizer, fr_tokenizer
    except OSError:
        print("Lỗi: Chưa tìm thấy model Spacy. Hãy chạy lại Bước 1 để tải en_core_web_sm/fr_core_news_sm.")
        return None, None
def build_vocab(filepath, tokenizer):
    def yield_tokens(path):
        with io.open(path, encoding='utf-8') as f:
            for line in f:
                if line.strip():
                    yield tokenizer(line.strip())
    print(f"Đang xây dựng vocab từ {filepath}...")

    vocab = build_vocab_from_iterator(
        yield_tokens(filepath),
        min_freq=2,
        max_tokens=10000,
        specials=SPECIAL_SYMBOLS
    )

    vocab.set_default_index(UNK_IDX)
    return vocab

# 3. THỰC THI (MAIN)
tokenizer_en, tokenizer_fr = get_tokenizers()
if tokenizer_en and tokenizer_fr:
    vocab_en = build_vocab('data/train.en', tokenizer_en)
    vocab_fr = build_vocab('data/train.fr', tokenizer_fr)
    # -----
    print("\n" + "="*40)
    print(" BÁO CÁO KẾT QUẢ")
    print("="*40)
    print(f"1. Kích thước từ điển EN: {len(vocab_en)} từ (Yêu cầu: <= 10004)")
    print(f"2. Kích thước từ điển FR: {len(vocab_fr)} từ (Yêu cầu: <= 10004)")
    print(f"3. Kiểm tra index token: <unk>={vocab_en['<unk>']}, <pad>={vocab_en['<pad>']}")

    # Kiểm tra xử lý từ lạ (OOV)
    test_oov = vocab_en['từ_này_chắc_chắn_không_có']
    if test_oov == UNK_IDX:
        print("4. Xử lý từ lạ (OOV):  Thành công (Trả về index 0)")
    else:
        print(f"4. Xử lý từ lạ (OOV): Thất bại (Trả về {test_oov})")
    print("="*40)

 Đã tải thành công Tokenizer (Spacy).
Đang xây dựng vocab từ data/train.en...
Đang xây dựng vocab từ data/train.fr...

 BÁO CÁO KẾT QUẢ
1. Kích thước từ điển EN: 6191 từ (Yêu cầu: <= 10004)
2. Kích thước từ điển FR: 6555 từ (Yêu cầu: <= 10004)
3. Kiểm tra index token: <unk>=0, <pad>=1
4. Xử lý từ lạ (OOV):  Thành công (Trả về index 0)


In [25]:

def text_transform(text, tokenizer, vocab):
    tokens = tokenizer(text)
    token_ids = [vocab[token] for token in tokens]
    return torch.tensor([SOS_IDX] + token_ids + [EOS_IDX], dtype=torch.long)

print("\n" + "="*40)
print(" KIỂM TRA TEXT PIPELINE")
print("="*40)
sample_sentence = "Two young, White males are outside."
print(f"1. Câu gốc: {sample_sentence}")

# Chạy qua hàm transform
sample_tensor = text_transform(sample_sentence, tokenizer_en, vocab_en)

print(f"2. Kết quả Tensor: {sample_tensor}")
print(f"3. Shape: {sample_tensor.shape}")
print(f"4. Kiểu dữ liệu: {sample_tensor.dtype}")

# Kiểm tra logic <sos> và <eos>
if sample_tensor[0] == SOS_IDX and sample_tensor[-1] == EOS_IDX:
    print("Logic <sos>/<eos>: Chính xác (Đầu là 2, Cuối là 3)")
else:
    print(f"Logic <sos>/<eos>:SAI (Đầu: {sample_tensor[0]}, Cuối: {sample_tensor[-1]})")
print("="*40)


 KIỂM TRA TEXT PIPELINE
1. Câu gốc: Two young, White males are outside.
2. Kết quả Tensor: tensor([   2,   19,   25,   15, 1169,  808,   17,   57,    5,    3])
3. Shape: torch.Size([10])
4. Kiểu dữ liệu: torch.int64
Logic <sos>/<eos>: Chính xác (Đầu là 2, Cuối là 3)


In [26]:
# ==============================================================================
# 1. CLASS DATASET
# ==============================================================================
class TranslationDataset(Dataset):
    def __init__(self, src_list, trg_list):
        self.src_list = src_list
        self.trg_list = trg_list

    def __len__(self):
        return len(self.src_list)

    def __getitem__(self, idx):

        return self.src_list[idx], self.trg_list[idx]

# ==============================================================================
# 2. HÀM COLLATE_FN
# ==============================================================================
def collate_fn(batch):
    src_batch, trg_batch = [], []

    # Chuyển đổi Text -> Tensor
    for src_sample, trg_sample in batch:
        src_batch.append(text_transform(src_sample, tokenizer_en, vocab_en))
        trg_batch.append(text_transform(trg_sample, tokenizer_fr, vocab_fr))

    # Padding (Đồng bộ độ dài)
    src_padded = pad_sequence(src_batch, padding_value=PAD_IDX)
    trg_padded = pad_sequence(trg_batch, padding_value=PAD_IDX)

    # Tính độ dài thực tế của từng câu nguồn
    src_lens = torch.tensor([len(x) for x in src_batch])

    # Sort giảm dần
    sorted_lens, sorted_indices = torch.sort(src_lens, descending=True)

    # Sắp xếp lại Tensor theo thứ tự index đã sort
    src_padded = src_padded[:, sorted_indices]
    trg_padded = trg_padded[:, sorted_indices]

    return src_padded, trg_padded, sorted_lens

# ==============================================================================
# 3. HÀM GET_DATA_LOADER
# ==============================================================================
def get_data_loader(dataset, batch_size, collate_fn, shuffle=False):
    return DataLoader(dataset,
                      batch_size=batch_size,
                      shuffle=shuffle,
                      collate_fn=collate_fn)

# ==============================================================================
# 4. THỰC THI & TẠO LOADER
# ==============================================================================
BATCH_SIZE = 64
train_dataset = TranslationDataset(train_en, train_fr)
valid_dataset = TranslationDataset(val_en, val_fr)
test_dataset  = TranslationDataset(test_en, test_fr)
train_loader = get_data_loader(train_dataset, BATCH_SIZE, collate_fn, shuffle=True)
valid_loader = get_data_loader(valid_dataset, BATCH_SIZE, collate_fn, shuffle=False)
test_loader  = get_data_loader(test_dataset,  BATCH_SIZE, collate_fn, shuffle=False)
# ==============================================================================
# 5. KIỂM TRA LOGIC MỚI
# ==============================================================================
print("\n" + "="*40)
print(" KIỂM TRA LOGIC MỚI")
print("="*40)

# Lấy 1 batch bất kỳ
try:
    src, trg, src_len = next(iter(train_loader))

    print(f"1. Shape Source: {src.shape}")
    print(f"2. Shape Target: {trg.shape}")
    print(f"3. Shape Lengths: {src_len.shape}")

    # Kiểm tra giá trị Length
    print(f"   - Length câu đầu tiên (Sau khi sort): {src_len[0]}")
    print(f"   - Length câu cuối cùng (Sau khi sort): {src_len[-1]}")

    # Kiểm tra Sorting
    if src_len[0] >= src_len[-1]:
        print("KẾT QUẢ: Batch đã sort + Có trả về Lengths.")
        print("   -> Sẵn sàng cho Encoder (pack_padded_sequence).")
    else:
        print(" LỖI: Sort chưa đúng.")

    print("="*40)
    print(f"Số batch trong train_loader: {len(train_loader)}")
    print(f"Số batch trong valid_loader: {len(valid_loader)}")
    print(f"Số batch trong test_loader: {len(test_loader)}")
    print("="*40)

except ValueError as e:
    print(" LỖI CẤU TRÚC: DataLoader không trả về đủ 3 giá trị!")
    print(e)


 KIỂM TRA LOGIC MỚI
1. Shape Source: torch.Size([34, 64])
2. Shape Target: torch.Size([34, 64])
3. Shape Lengths: torch.Size([64])
   - Length câu đầu tiên (Sau khi sort): 34
   - Length câu cuối cùng (Sau khi sort): 9
KẾT QUẢ: Batch đã sort + Có trả về Lengths.
   -> Sẵn sàng cho Encoder (pack_padded_sequence).
Số batch trong train_loader: 454
Số batch trong valid_loader: 16
Số batch trong test_loader: 16


## PHẦN 2: XÂY DỰNG MÔ HÌNH ENCODER-DECODER LSTM


- Encoder: `(ht, ct) = LSTM(embed(xt), (ht-1, ct-1))`
- Decoder: `(ht, ct) = LSTM(embed(yt-1), (h't-1, c't-1))` và `p(yt) = softmax(Linear(ht))`
- Context vector cố định từ Encoder (không bắt buộc attention)

**Tham số khuyến nghị:**
- Hidden size: 512
- Embedding dim: 256–512
- Số layer LSTM: 2
- Dropout: 0.3–0.5

In [27]:
# ==============================================================================
# 1. ENCODER
# ==============================================================================
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_len):
        embedded = self.dropout(self.embedding(src))
        packed_embedded = pack_padded_sequence(embedded, src_len.cpu(), enforce_sorted=True)
        packed_outputs, (hidden, cell) = self.lstm(packed_embedded)
        return hidden, cell

# ==============================================================================
# 2. DECODER
# ==============================================================================
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

# ==============================================================================
# 3. SEQ2SEQ
# ==============================================================================
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must match!"
        assert encoder.n_layers == decoder.n_layers, \
            "Number of layers of encoder and decoder must match!"

    def forward(self, src, src_len, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src, src_len)
        input = trg[0,:]


        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs

def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.uniform_(param.data, -0.08, 0.08)
        else:
            nn.init.constant_(param.data, 0)

print("Đã xây dựng xong kiến trúc Model (Encoder - Decoder - Seq2Seq).")

Đã xây dựng xong kiến trúc Model (Encoder - Decoder - Seq2Seq).


In [28]:

# 1. Cấu hình Hyperparameters
INPUT_DIM = len(vocab_en)
OUTPUT_DIM = len(vocab_fr)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

# Thiết bị
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Khởi tạo Model
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)
model.apply(init_weights)

print(f" Đã khởi tạo Model trên: {device}")
print(f" Tổng số tham số: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# 3. TEST KẾT NỐI VỚI DATALOADER (QUAN TRỌNG)
print("\n" + "="*40)
print(" KIỂM TRA KẾT NỐI DATA -> MODEL")
print("="*40)

try:
    batch = next(iter(train_loader))
    src, trg, src_len = batch
    src = src.to(device)
    trg = trg.to(device)

    print(f"1. Input Shape (Src): {src.shape}")
    print(f"2. Input Lengths (Len): {src_len.shape}")
    print(f"3. Target Shape (Trg): {trg.shape}")
    output = model(src, src_len, trg)
    print(f"4. Output Shape: {output.shape}")

    # Kiểm tra kích thước
    if output.shape[0] == trg.shape[0] and output.shape[1] == trg.shape[1] and output.shape[2] == OUTPUT_DIM:
        print("\n THÀNH CÔNG")
        print("    Sẵn sàng cho quá trình Huấn luyện .")
    else:
        print("\n CẢNH BÁO: Kích thước đầu ra không khớp!")

except Exception as e:
    print(f"\n LỖI KHI CHẠY THỬ: {e}")
    print("Gợi ý: Kiểm tra lại xem collate_fn ở Phần 1 có trả về đúng 3 giá trị không?")


 Đã khởi tạo Model trên: cpu
 Tổng số tham số: 13,982,107

 KIỂM TRA KẾT NỐI DATA -> MODEL
1. Input Shape (Src): torch.Size([30, 64])
2. Input Lengths (Len): torch.Size([64])
3. Target Shape (Trg): torch.Size([33, 64])
4. Output Shape: torch.Size([33, 64, 6555])

 THÀNH CÔNG
    Sẵn sàng cho quá trình Huấn luyện .
